In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from src import EnergyTrading, GroupDistributor

In [2]:
# file_path = "data/meter_month/apt_780_201809.csv"
file_path = "data/meter_month/apt_1_month_9.csv"

trade = EnergyTrading(file_path, month=9)
dist = GroupDistributor(file_path).set_init_csv(month=9)

households_kwh = dist.meter_month['usage (kWh)'].sum()
public_kwh = households_kwh * 0.25
apt = households_kwh + public_kwh

### EneryTrading

In [12]:
trade.set_init()
trade.simulate()
trade.set_settlement_infos()
sim_df = trade.simulation_datas

In [13]:
trade.datas.head(10)

,name,usage (kWh),전력량요금,공용부요금
0,아파트1-104-1206,172.980,13527.0360,6901.500402
1,아파트1-104-303,6.609,516.8238,6901.500402
2,아파트1-104-1307,98.065,7668.6830,6901.500402
3,아파트1-104-1208,6.302,492.8164,6901.500402
4,아파트1-104-408,41.189,3220.9798,6901.500402
5,아파트1-104-203,230.701,20159.1872,6901.500402
6,아파트1-103-1402,213.530,17631.6160,6901.500402
7,아파트1-103-402,239.237,21415.6864,6901.500402
8,아파트1-103-1201,182.576,14277.4432,6901.500402
9,아파트1-103-801,193.109,15101.1238,6901.500402


In [5]:
sim_df.sort_values(by=["차이"], ascending=False).head(10)

,name,usage (kWh),전력량 요금,거래 전력량요금(거래가격 적용 X),공용부요금(거래 적용 X),거래 전력량요금(거래 적용 O),공용부요금(거래 적용 O),차이
74,아파트1-104-1006,200.0,0.1564,15640.0,8482.8508,-13210.923823,8482.8508,13211.080223
3,아파트1-104-1208,200.0,492.8164,15640.0,8482.8508,-12302.110634,8482.8508,12794.927034
1,아파트1-104-303,200.0,516.8238,15640.0,8482.8508,-12257.824023,8482.8508,12774.647823
75,아파트1-104-902,200.0,591.0356,15640.0,8482.8508,-12120.925021,8482.8508,12711.960621
35,아파트1-101-704,200.0,2566.7586,15640.0,8482.8508,-8476.295623,8482.8508,11043.054223
4,아파트1-104-408,200.0,3220.9798,15640.0,8482.8508,-7269.449411,8482.8508,10490.429211
67,아파트1-104-404,200.0,6281.4150,15640.0,8482.8508,-1623.844181,8482.8508,7905.259181
26,아파트1-101-1604,200.0,7117.6858,15640.0,8482.8508,-81.169857,8482.8508,7198.855657
69,아파트1-104-905,200.0,7513.4560,15640.0,8482.8508,648.910071,8482.8508,6864.545929
2,아파트1-104-1307,200.0,7668.6830,15640.0,8482.8508,935.258353,8482.8508,6733.424647


### Distributor

In [6]:
dist.set_calc(kwh=apt)
dist.set_cont()
dist_df = dist.distribute_table()

dist_df.drop_duplicates('기여도').sort_values(['기여도'])

,가구명,공용부요금,기여도,기여도 기본요금,기여도 전력량요금,나머지 요금,기여도 요금,오차
1,아파트1-104-303,8633,0.020409,0.0,140.0,7620.259259,7760.0,873.0
2,아파트1-104-1307,8633,0.061225,0.0,421.0,7620.259259,8041.0,592.0
0,아파트1-104-1206,8633,0.102041,0.0,702.0,7620.259259,8322.0,311.0
5,아파트1-104-203,8633,0.142857,1.0,983.0,7620.259259,8604.0,29.0
10,아파트1-103-502,8633,0.183673,1.0,1264.0,7620.259259,8885.0,-252.0
13,아파트1-103-1502,8633,0.224489,1.0,1545.0,7620.259259,9166.0,-533.0
43,아파트1-102-802,8633,0.265305,1.0,1826.0,7620.259259,9447.0,-814.0


### Distributor in min-max-normalization

In [7]:
from src.utils import min_max_normalization

In [8]:
dist.set_calc(kwh=apt)
dist.set_cont(norm=min_max_normalization)
dist_df = dist.distribute_table()

dist_df.drop_duplicates('기여도').sort_values(['기여도'])

,가구명,공용부요금,기여도,기여도 기본요금,기여도 전력량요금,나머지 요금,기여도 요금,오차
1,아파트1-104-303,8633,0.0,0.0,0.0,5073.074074,5073.0,3560.0
2,아파트1-104-1307,8633,0.166667,1.0,1147.0,5073.074074,6221.0,2412.0
0,아파트1-104-1206,8633,0.333333,1.0,2294.0,5073.074074,7368.0,1265.0
5,아파트1-104-203,8633,0.5,2.0,3441.0,5073.074074,8516.0,117.0
10,아파트1-103-502,8633,0.666667,3.0,4587.0,5073.074074,9663.0,-1030.0
13,아파트1-103-1502,8633,0.833333,3.0,5734.0,5073.074074,10810.0,-2177.0
43,아파트1-102-802,8633,1.0,4.0,6881.0,5073.074074,11958.0,-3325.0
